In [4]:
#Importing packages
!{sys.executable} -m pip install nba_api
from nba_api.stats.static import players
# from nba_api.stats.endpoints import leaguegamefinder
# from nba_api.stats.endpoints import commonplayerinfo
# from nba_api.stats.endpoints import playbyplayv2
import requests, pandas as pd, sys
# !{sys.executable} -m pip install tqdm
from tqdm import tqdm

Using legacy 'setup.py install' for nba-api, since package 'wheel' is not installed.
    Running setup.py install for nba-api: started
    Running setup.py install for nba-api: finished with status 'done'


You should consider upgrading via the 'c:\Users\zman7\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
#Creating header var for api pull
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [4]:
# create function that gets pbp (play by play) logs from the 2020-21 season
def get_data(game_id):
    play_by_play_url = "https://cdn.nba.com/static/json/liveData/playbyplay/playbyplay_"+game_id+".json"
    response = requests.get(url=play_by_play_url, headers=headers).json()
    play_by_play = response['game']['actions']
    df = pd.DataFrame(play_by_play)
    df['gameid'] = game_id
    return df

In [5]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2020-21',
                                               league_id_nullable='00',
                                               season_type_nullable='Regular Season')

In [6]:
games = gamefinder.get_data_frames()[0]

In [7]:
#getting a list of distinct game ids
game_ids = games['GAME_ID'].unique()

In [8]:
len(game_ids)

1080

In [9]:
pbpdata = []

In [10]:
#Here TDQM() creates a % completion bar
for game_id in tqdm(game_ids):
    game_data = get_data(game_id)
    pbpdata.append(game_data)

final_df = pd.concat(pbpdata, ignore_index=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1080/1080 [04:22<00:00,  4.11it/s]


In [20]:
#pbpdata

In [15]:
final_df.size

33285890

In [16]:
final_df_copy = pd.DataFrame(final_df)

In [17]:
#Primary columns in the DF
final_df_copy.columns

Index(['actionNumber', 'clock', 'timeActual', 'period', 'periodType',
       'actionType', 'subType', 'qualifiers', 'personId', 'x', 'y',
       'possession', 'scoreHome', 'scoreAway', 'edited', 'orderNumber',
       'xLegacy', 'yLegacy', 'isFieldGoal', 'side', 'description',
       'personIdsFilter', 'teamId', 'teamTricode', 'descriptor',
       'jumpBallRecoveredName', 'jumpBallRecoverdPersonId', 'playerName',
       'playerNameI', 'jumpBallWonPlayerName', 'jumpBallWonPersonId',
       'jumpBallLostPlayerName', 'jumpBallLostPersonId', 'shotDistance',
       'shotResult', 'shotActionNumber', 'reboundTotal',
       'reboundDefensiveTotal', 'reboundOffensiveTotal', 'pointsTotal',
       'assistPlayerNameInitial', 'assistPersonId', 'assistTotal',
       'blockPlayerName', 'blockPersonId', 'officialId', 'foulPersonalTotal',
       'foulTechnicalTotal', 'foulDrawnPlayerName', 'foulDrawnPersonId',
       'turnoverTotal', 'stealPlayerName', 'stealPersonId', 'gameid', 'value'],
      dtype='o

In [18]:
final_df_copy.subType.unique()

array(['start', 'recovered', 'Jump Shot', 'defensive', 'Layup', nan,
       'offensive', 'out-of-bounds', 'personal', '1 of 2', '2 of 2',
       'bad pass', 'lost ball', 'DUNK', 'kicked ball', 'full', 'out',
       'in', 'Hook', 'end', 'traveling', '1 of 1', 'equipment issue',
       'palming', '1 of 3', '2 of 3', '3 of 3', 'offensive foul',
       'technical', 'shot clock', 'defensive goaltending', 'request',
       'challenge', 'delay-of-game', 'other', '3-second-violation',
       'blood rule', 'altercationrequest', 'discontinued dribble',
       'injury', 'backcourt', 'offensive-kicked-ball', 'jumpball',
       'offensive goaltending', 'LaneViolation', '5-second-violation',
       'lane', 'double dribble', 'court clean up', 'inbound',
       '8-second-violation', 'double lane', 'jumpball violation',
       '5-second-back-to-the-basket', 'shot', 'swinging elbows',
       'illegal assist', 'excess timeout', 'illegal-out-of-bounds-screen'],
      dtype=object)

In [19]:
pd.set_option('display.max_columns', None)

In [20]:
final_df_copy.head(6)

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,y,possession,scoreHome,scoreAway,edited,orderNumber,xLegacy,yLegacy,isFieldGoal,side,description,personIdsFilter,teamId,teamTricode,descriptor,jumpBallRecoveredName,jumpBallRecoverdPersonId,playerName,playerNameI,jumpBallWonPlayerName,jumpBallWonPersonId,jumpBallLostPlayerName,jumpBallLostPersonId,shotDistance,shotResult,shotActionNumber,reboundTotal,reboundDefensiveTotal,reboundOffensiveTotal,pointsTotal,assistPlayerNameInitial,assistPersonId,assistTotal,blockPlayerName,blockPersonId,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,turnoverTotal,stealPlayerName,stealPersonId,gameid,value
0,2,PT12M00.00S,2021-05-16T17:10:26.7Z,1,REGULAR,period,start,[],0,NaN,NaN,0,0,0,2021-05-16T17:10:26Z,20000,NaN,NaN,0,None,Period Start,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
1,4,PT11M58.00S,2021-05-16T17:10:28.6Z,1,REGULAR,jumpball,recovered,[],203926,NaN,NaN,1610612754,0,0,2021-05-16T17:10:28Z,40000,NaN,NaN,0,None,Jump Ball D. Sabonis vs. F. Gillespie: Tip to ...,"[203926, 1627734, 1630273]",1.610613e+09,IND,startperiod,D. McDermott,203926.0,McDermott,D. McDermott,Sabonis,1627734.0,Gillespie,1630273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
2,7,PT11M42.00S,2021-05-16T17:10:44.6Z,1,REGULAR,3pt,Jump Shot,[],1629052,28.728647,22.617953,1610612754,0,0,2021-05-16T17:10:54Z,70000,137.0,218.0,1,left,MISS O. Brissett 25' 3PT,[1629052],1.610613e+09,IND,NaN,NaN,NaN,Brissett,O. Brissett,NaN,NaN,NaN,NaN,25.70,Missed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
3,8,PT11M35.00S,2021-05-16T17:10:51.6Z,1,REGULAR,rebound,defensive,[],1626169,NaN,NaN,1610612761,0,0,2021-05-16T17:10:54Z,80000,NaN,NaN,0,None,S. Johnson REBOUND (Off:0 Def:1),[1626169],1.610613e+09,TOR,NaN,NaN,NaN,Johnson,S. Johnson,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
4,9,PT11M29.00S,2021-05-16T17:10:57.2Z,1,REGULAR,2pt,Layup,[pointsinthepaint],1627761,92.591984,57.176777,1610612761,2,0,2021-05-16T17:11:02Z,90000,36.0,17.0,1,right,D. Bembry driving Layup (2 PTS) (K. Birch 1 AST),"[1627761, 203920]",1.610613e+09,TOR,driving,NaN,NaN,Bembry,D. Bembry,NaN,NaN,NaN,NaN,3.98,Made,NaN,NaN,NaN,NaN,2.0,K. Birch,203920.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
5,11,PT11M17.00S,2021-05-16T17:11:10.1Z,1,REGULAR,2pt,Layup,[pointsinthepaint],203200,9.806176,49.088542,1610612754,2,0,2021-05-16T17:11:21Z,110000,5.0,40.0,1,left,MISS J. Holiday driving Layup - blocked,"[203200, 1630201]",1.610613e+09,IND,driving,NaN,NaN,Holiday,J. Holiday,NaN,NaN,NaN,NaN,4.00,Missed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flynn,1630201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN


In [21]:
#Notes for to dos for this data set
--Look at % of makes of shots based on distance
--What duo had the most assits to dunks in the NBA
--Calculate true shooting % of all players? using apply? or creating a function... probably this
    #TS% - True Shooting Percentage; the formula is PTS / (2 * TSA). True shooting percentage is a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.
    #TSA - True Shooting Attempts; the formula is FGA + 0.44 * FTA.

SyntaxError: invalid syntax (2600496450.py, line 2)

In [22]:
#Looking at shots taken from rounded distances
final_df_copy.loc[:,['shotDistance']].round().value_counts()#.sort_index()

shotDistance
26.0            19638
25.0            17866
2.0             17536
1.0             16470
3.0             11599
                ...  
79.0                1
80.0                1
81.0                1
86.0                1
87.0                1
Length: 82, dtype: int64

In [23]:
#breaking down shot distance and the % of total shots made from the distance
final_df_copy.loc[:, ['shotResult']].value_counts()

shotResult
Made          125670
Missed        112435
dtype: int64

In [24]:
final_df_copy['actionType'].unique()

array(['period', 'jumpball', '3pt', 'rebound', '2pt', 'block', 'stoppage',
       'foul', 'freethrow', 'turnover', 'steal', 'violation', 'timeout',
       'substitution', 'game', 'instantreplay', 'ejection', 'memo'],
      dtype=object)

In [25]:
#pulling rows where actionType = 'block'
final_df_copy[final_df_copy.actionType == 'block']

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,y,possession,scoreHome,scoreAway,edited,orderNumber,xLegacy,yLegacy,isFieldGoal,side,description,personIdsFilter,teamId,teamTricode,descriptor,jumpBallRecoveredName,jumpBallRecoverdPersonId,playerName,playerNameI,jumpBallWonPlayerName,jumpBallWonPersonId,jumpBallLostPlayerName,jumpBallLostPersonId,shotDistance,shotResult,shotActionNumber,reboundTotal,reboundDefensiveTotal,reboundOffensiveTotal,pointsTotal,assistPlayerNameInitial,assistPersonId,assistTotal,blockPlayerName,blockPersonId,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,turnoverTotal,stealPlayerName,stealPersonId,gameid,value
6,12,PT11M17.00S,2021-05-16T17:11:10.1Z,1,REGULAR,block,NaN,[],1630201,NaN,NaN,1610612754,2,0,2021-05-16T17:11:12Z,120000,NaN,NaN,0,None,M. Flynn BLOCK (1 BLK),[1630201],1.610613e+09,TOR,NaN,NaN,NaN,Flynn,M. Flynn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
152,206,PT09M33.00S,2021-05-16T17:40:37.3Z,2,REGULAR,block,NaN,[],203382,NaN,NaN,1610612754,32,37,2021-05-16T17:43:16Z,1837500,NaN,NaN,0,None,A. Baynes BLOCK (1 BLK),[203382],1.610613e+09,TOR,NaN,NaN,NaN,Baynes,A. Baynes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
174,222,PT07M51.00S,2021-05-16T17:43:50.1Z,2,REGULAR,block,NaN,[],203382,NaN,NaN,1610612754,41,42,2021-05-16T17:44:47Z,2075000,NaN,NaN,0,None,A. Baynes BLOCK (2 BLK),[203382],1.610613e+09,TOR,NaN,NaN,NaN,Baynes,A. Baynes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
311,380,PT09M24.00S,2021-05-16T18:26:10.1Z,3,REGULAR,block,NaN,[],204456,NaN,NaN,1610612761,64,74,2021-05-16T18:26:13Z,3730000,NaN,NaN,0,None,T. McConnell BLOCK (1 BLK),[204456],1.610613e+09,IND,NaN,NaN,NaN,McConnell,T. McConnell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
467,557,PT07M33.00S,2021-05-16T18:55:51.5Z,4,REGULAR,block,NaN,[],203960,NaN,NaN,1610612761,91,103,2021-05-16T18:55:53Z,5500000,NaN,NaN,0,None,J. Sampson BLOCK (1 BLK),[203960],1.610613e+09,IND,NaN,NaN,NaN,Sampson,J. Sampson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022001079,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604903,371,PT11M47.00S,2020-12-23T01:20:43.7Z,3,REGULAR,block,NaN,[],201142,NaN,NaN,1610612744,63,45,2020-12-23T01:20:46Z,3550000,NaN,NaN,0,None,K. Durant BLOCK (1 BLK),[201142],1.610613e+09,BKN,NaN,NaN,NaN,Durant,K. Durant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022000001,NaN
604928,426,PT10M04.00S,2020-12-23T01:25:12.5Z,3,REGULAR,block,NaN,[],1626162,NaN,NaN,1610612751,69,49,2020-12-23T01:31:00Z,3837500,NaN,NaN,0,None,K. Oubre Jr. BLOCK (1 BLK),[1626162],1.610613e+09,GSW,NaN,NaN,NaN,Oubre Jr.,K. Oubre Jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022000001,NaN
604943,482,PT08M41.00S,2020-12-23T01:28:04.7Z,3,REGULAR,block,NaN,[],203952,NaN,NaN,1610612751,75,49,2020-12-23T01:38:14Z,4025000,NaN,NaN,0,None,A. Wiggins BLOCK (1 BLK),[203952],1.610613e+09,GSW,NaN,NaN,NaN,Wiggins,A. Wiggins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022000001,NaN
604995,527,PT05M00.00S,2020-12-23T01:38:13.6Z,3,REGULAR,block,NaN,[],1626162,NaN,NaN,1610612751,86,58,2020-12-23T01:46:47Z,4625000,NaN,NaN,0,None,K. Oubre Jr. BLOCK (2 BLK),[1626162],1.610613e+09,GSW,NaN,NaN,NaN,Oubre Jr.,K. Oubre Jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022000001,NaN


In [26]:
#take shotDistance, actionType is 2pt and 3pt, shotResult is Missed or Made
all_actions = pd.DataFrame(final_df_copy.loc[:, ["shotDistance", "actionType", "shotResult"]])

In [27]:
all_actions

,shotDistance,actionType,shotResult
0,NaN,period,NaN
1,NaN,jumpball,NaN
2,25.70,3pt,Missed
3,NaN,rebound,NaN
4,3.98,2pt,Made
...,...,...,...
605193,8.62,2pt,Missed
605194,NaN,rebound,NaN
605195,25.91,3pt,Made
605196,NaN,period,NaN


In [28]:
shots_df = all_actions[(all_actions.shotResult.isin(['Missed', 'Made'])) & (all_actions.actionType.isin(['2pt', '3pt']))]

In [29]:
shots_df

,shotDistance,actionType,shotResult
2,25.70,3pt,Missed
4,3.98,2pt,Made
5,4.00,2pt,Missed
9,22.76,3pt,Missed
12,6.62,2pt,Made
...,...,...,...
605182,25.27,3pt,Missed
605184,9.65,2pt,Missed
605186,0.00,2pt,Made
605193,8.62,2pt,Missed


In [44]:
shots_df.sort_index(axis = 1, inplace = True)
shots_df = shots_df.round()

In [45]:
#shots_df.shotResult.replace(['Missed', 0], ['Made', 1])
shots_df['shotResult'] = shots_df.shotResult.replace(['Missed','Made'], [0, 1])

In [52]:
shots_df.shotResult.value_counts()

0    101950
1     89020
Name: shotResult, dtype: int64

In [58]:
def percentcalc(x):
    for i in shots_df[shots_df.shotResult]:
        if shots_df[shots_df.shotResult == 'Made']:
            return 1
        else:
            return 0

In [59]:
shots_df.applymap(percentcalc)

KeyError: "None of [Int64Index([0, 1, 0, 0, 1, 0, 1, 0, 1, 0,\n            ...\n            0, 1, 0, 0, 1, 0, 0, 1, 0, 1],\n           dtype='int64', length=190970)] are in the [columns]"

In [57]:
shots_grouped.shotResult.sum()

190970

In [61]:
#looking into what player has assisted the most dunks in the nba (or per team)
assists = final_df[(final_df.subType == 'DUNK') & (final_df.periodType == 'REGULAR')].loc[:,['assistPlayerNameInitial','playerNameI','teamTricode', 'teamId' ]]

In [62]:
#adding new col with combined player
assists = assists[pd.notnull(assists.assistPlayerNameInitial)]


In [63]:
assists['assistor-assiste'] = assists.assistPlayerNameInitial + '  to  ' + assists.playerNameI  

In [1]:
assists.columns()

NameError: name 'assists' is not defined

In [76]:
assistsfinal = assists.loc[: ,['teamTricode','assistor-assiste']].sort_values(by=['teamTricode'])

In [88]:
assists_grouped = assistsfinal.groupby(['teamTricode','assistor-assiste'])['assistor-assiste'].count()

In [90]:
assists_grouped

teamTricode  assistor-assiste              
ATL          B. Bogdanovic  to  C. Capela      10
             B. Bogdanovic  to  J. Collins      5
             B. Bogdanovic  to  K. Huerter      1
             B. Bogdanovic  to  N. Knight       3
             B. Bogdanovic  to  O. Okongwu      2
                                               ..
WAS          R. Westbrook  to  R. Hachimura    31
             R. Westbrook  to  R. Lopez        13
             R. Westbrook  to  T. Bryant        6
             T. Brown Jr.  to  R. Westbrook     1
             T. Brown Jr.  to  T. Bryant        1
Name: assistor-assiste, Length: 2248, dtype: int64